In [1]:
## file folder
import os

folder_path = '/storage/users/sac43cg/res_Samantha_1/outs/per_sample_outs'

# Get the list of files and directories in the folder
folder_content = os.listdir(folder_path)

# Print the content of the folder
for item in folder_content:
    print(item)

CTRL_1
CTRL_2
CTRL_2D
GEM_2
TGFb1_1
TGFb1_2
TGFb1_GEM_1
TGFb1_GEM_2


In [2]:
import os
import gzip

In [3]:
def unzip_all_gz_files(folder_path):
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        

        if file_name.endswith('.gz'):
            unzipped_file_path = os.path.splitext(file_path)[0]  # Remove the .gz extension
            #print(file_path)
            with gzip.open(file_path, 'rb') as gz_file:
                with open(unzipped_file_path, 'wb') as unzipped_file:
                    unzipped_file.write(gz_file.read())
            
            # Optionally, you can delete the .gz file after unzipping
            # os.remove(file_path)


samples = folder_content
for sample in samples:

    folder_path = f'/storage/users/sac43cg/res_Samantha_1/outs/per_sample_outs/{sample}/count/sample_filtered_feature_bc_matrix'
    df = unzip_all_gz_files(folder_path)

    #print(df.head())
    print(folder_path)

/storage/users/sac43cg/res_Samantha_1/outs/per_sample_outs/CTRL_1/count/sample_filtered_feature_bc_matrix
/storage/users/sac43cg/res_Samantha_1/outs/per_sample_outs/CTRL_2/count/sample_filtered_feature_bc_matrix
/storage/users/sac43cg/res_Samantha_1/outs/per_sample_outs/CTRL_2D/count/sample_filtered_feature_bc_matrix
/storage/users/sac43cg/res_Samantha_1/outs/per_sample_outs/GEM_2/count/sample_filtered_feature_bc_matrix
/storage/users/sac43cg/res_Samantha_1/outs/per_sample_outs/TGFb1_1/count/sample_filtered_feature_bc_matrix
/storage/users/sac43cg/res_Samantha_1/outs/per_sample_outs/TGFb1_2/count/sample_filtered_feature_bc_matrix
/storage/users/sac43cg/res_Samantha_1/outs/per_sample_outs/TGFb1_GEM_1/count/sample_filtered_feature_bc_matrix
/storage/users/sac43cg/res_Samantha_1/outs/per_sample_outs/TGFb1_GEM_2/count/sample_filtered_feature_bc_matrix


In [4]:
#https://support.10xgenomics.com/single-cell-gene-expression/software/pipelines/latest/output/matrices
import scipy.io as sio
import os
import anndata as ad
import scanpy as sc
import pandas as pd


In [5]:
##samples = ['sample1', 'sample2']  # Add more sample names if needed
conditions = folder_content
#pwd /storage/users/sac43cg/res_Samantha/outs/per_sample_outs/CTRL_1/count/sample_filtered_feature_bc_matrix

# Create an empty list to store the AnnData objects
adata_list = []
barcodes_list = []
features_list = []
matrix_list = []
#combined_adata = []

for condition in conditions:
           
    # Define the file paths
    base_path = f'/storage/users/sac43cg/res_Samantha_1/outs/per_sample_outs/{condition}/count/sample_filtered_feature_bc_matrix/'  # Replace with the actual path to your data

    barcodes_file = os.path.join(base_path, 'barcodes.tsv')
    features_file = os.path.join(base_path, 'features.tsv')
    matrix_file = os.path.join(base_path, 'matrix.mtx')

    # Read the barcodes.tsv file
    barcodes = pd.read_csv(barcodes_file, header=None, index_col=0, names=['barcode'])

    # Read the features.tsv file
    features = pd.read_csv(features_file, sep='\t', header=None, index_col=0, names=['feature_id', 'symbol', 'type'])
    # get only gene expression features
    #features = features[features['type'] == 'gene expression']

    # Read the matrix.mtx file
    matrix = sio.mmread(matrix_file)

    # Create the AnnData object
    adata = ad.AnnData(X=matrix.T, obs=barcodes, var=features)
    
    # Add a column to the observation metadata to indicate the condition
    adata.obs['condition'] = condition

    # Append the AnnData object to the list
    adata_list.append(adata)
    features['condition'] = condition
    features_list.append(features)
        
    
# Combine all the AnnData objects into a single AnnData object
combined_adata = ad.concat(adata_list, join='outer')




In [6]:
combined_adata.write('/storage/users/sac43cg/res_Samantha_1/outs/h5as_files/combined_adata.h5ad')


In [7]:
combined_adata

AnnData object with n_obs × n_vars = 11919 × 36613
    obs: 'condition'

In [8]:
# View the count matrix
print("Count matrix:")
print(combined_adata.X)

# View the observations (cells) information
print("\nObservations (Cells):")
print(combined_adata.obs)

# View the variables (genes) information
print("\nVariables (Genes):")
print(combined_adata.var)

Count matrix:
  (0, 29)	3
  (0, 58)	1
  (0, 59)	2
  (0, 62)	4
  (0, 73)	1
  (0, 86)	2
  (0, 94)	1
  (0, 97)	6
  (0, 99)	1
  (0, 104)	4
  (0, 105)	1
  (0, 132)	1
  (0, 142)	1
  (0, 169)	2
  (0, 170)	46
  (0, 190)	1
  (0, 194)	1
  (0, 202)	2
  (0, 208)	2
  (0, 211)	2
  (0, 216)	1
  (0, 219)	11
  (0, 227)	1
  (0, 238)	2
  (0, 241)	1
  :	:
  (11918, 36432)	1
  (11918, 36433)	3
  (11918, 36443)	1
  (11918, 36513)	1
  (11918, 36559)	22
  (11918, 36560)	21
  (11918, 36561)	27
  (11918, 36562)	50
  (11918, 36563)	11
  (11918, 36564)	21
  (11918, 36565)	27
  (11918, 36566)	64
  (11918, 36567)	2
  (11918, 36568)	43
  (11918, 36569)	9
  (11918, 36570)	2
  (11918, 36571)	14
  (11918, 36601)	76
  (11918, 36602)	103
  (11918, 36604)	46
  (11918, 36605)	509
  (11918, 36606)	41
  (11918, 36607)	22
  (11918, 36608)	6096
  (11918, 36609)	801

Observations (Cells):
                      condition
barcode                        
AAACCCAGTAGGCTCC-1       CTRL_1
AAACCCAGTCGCAACC-1       CTRL_1
AAACCCATCCACA

In [9]:
##### OPTIONAL ######
# here you can check your generated h5ad
# Print the annotations
print("Observation annotations:")
print(combined_adata.obs.head())  # Print the first few rows of observation annotations

print("\nVariable annotations:")
print(combined_adata.var.head())  # Print the first few rows of variable annotations

print("\nVariable annotations:")
print(combined_adata.varm)  # Print the first few rows of variable annotations

print("\nVariable features:")
print(features.head())
print(features['type'].unique())

# Print the dimensions
print("\nNumber of observations (cells):", combined_adata.n_obs)
print("Number of variables (features):", combined_adata.n_vars)


Observation annotations:
                   condition
barcode                     
AAACCCAGTAGGCTCC-1    CTRL_1
AAACCCAGTCGCAACC-1    CTRL_1
AAACCCATCCACAAGT-1    CTRL_1
AAACGCTTCACTACGA-1    CTRL_1
AAACGCTTCAGCCTCT-1    CTRL_1

Variable annotations:
Empty DataFrame
Columns: []
Index: [ENSG00000243485, ENSG00000237613, ENSG00000186092, ENSG00000238009, ENSG00000239945]

Variable annotations:
AxisArrays with keys: 

Variable features:
                      symbol             type    condition
feature_id                                                
ENSG00000243485  MIR1302-2HG  Gene Expression  TGFb1_GEM_2
ENSG00000237613      FAM138A  Gene Expression  TGFb1_GEM_2
ENSG00000186092        OR4F5  Gene Expression  TGFb1_GEM_2
ENSG00000238009   AL627309.1  Gene Expression  TGFb1_GEM_2
ENSG00000239945   AL627309.3  Gene Expression  TGFb1_GEM_2
['Gene Expression' 'Multiplexing Capture']

Number of observations (cells): 11919
Number of variables (features): 36613


In [10]:
# Iterate over each condition
for condition in combined_adata.obs['condition'].unique():
    condition_adata = combined_adata[combined_adata.obs['condition'] == condition]
    
    # Get the number of features and barcodes for the current condition
    num_features = condition_adata.n_vars
    num_barcodes = condition_adata.n_obs
    
    print("Observation annotations:")
    print(condition_adata.obs.head())  # Print the first few rows of observation annotations
    
    print("\nVariable annotations:")
    print(condition_adata.var.head())  # Print the first few rows of variable annotations
    
    # Write the condition name, number of features, and number of barcodes to the file
    print(f"Condition: {condition}\n")
    print(f"Number of features: {num_features}\n")
    print(f"Number of barcodes: {num_barcodes}\n\n")

Observation annotations:
                   condition
barcode                     
AAACCCAGTAGGCTCC-1    CTRL_1
AAACCCAGTCGCAACC-1    CTRL_1
AAACCCATCCACAAGT-1    CTRL_1
AAACGCTTCACTACGA-1    CTRL_1
AAACGCTTCAGCCTCT-1    CTRL_1

Variable annotations:
Empty DataFrame
Columns: []
Index: [ENSG00000243485, ENSG00000237613, ENSG00000186092, ENSG00000238009, ENSG00000239945]
Condition: CTRL_1

Number of features: 36613

Number of barcodes: 2973


Observation annotations:
                   condition
barcode                     
AAAGGATGTAGGAGTC-1    CTRL_2
AAAGGGCCACATACGT-1    CTRL_2
AAAGGGCGTTGAGAGC-1    CTRL_2
AAAGGTACAGAGCTAG-1    CTRL_2
AAAGGTAGTAGAGACC-1    CTRL_2

Variable annotations:
Empty DataFrame
Columns: []
Index: [ENSG00000243485, ENSG00000237613, ENSG00000186092, ENSG00000238009, ENSG00000239945]
Condition: CTRL_2

Number of features: 36613

Number of barcodes: 450


Observation annotations:
                   condition
barcode                     
AAACCCAAGAGGTATT-1   CTRL_2D

In [11]:
dimensions = barcodes.shape
print('\n' , dimensions)
dimensions = features.shape
print('\n' , dimensions)
dimensions = combined_adata.shape
print('\n' , dimensions)
dimensions = matrix.shape
print('\n' , dimensions)


 (485, 0)

 (36613, 3)

 (11919, 36613)

 (36613, 485)


In [12]:
print(combined_adata)

AnnData object with n_obs × n_vars = 11919 × 36613
    obs: 'condition'
